# Storage

These are very important opcodes because they manipulate storage, which is very expensive.

#### Storage Load

Load a `word` from storage given by a `key` and put it on the stack.

In [2]:
def sload(evm):
    key=evm.stack.pop()
    warm,value=evm.storage.load(key)
    evm.stack.push(value)


    # gas cost depends on warm/cold
    if warm:
        evm.gas_dec(100)
    else:
        evm.gas_dec(2100)

    evm.pc += 1

#### Storage Store

Get the `key` and the `word` that will be stored from the stack and store in storage.

In [3]:
def sstore(evm):
    key,value=evm.stack.pop(),evm.stack.pop()
    warm,old_value=evm.storage.load(key)

    base_dynamic_gas=0
    evm.storage.store(key, value)

    if value == old_value:
        base_dynamic_gas = 0
    elif old_value == 0 and value != 0:
        base_dynamic_gas = 20000
    else:
        base_dynamic_gas = 5000 

    access_cost = 100 if warm else 2100


    evm.gas_dec(base_dynamic_gas + access_cost)

    evm.pc += 1
   # TODO: do refunds 